# Проект по SQL 

Компания приобрела крупный сервис по чтению книг по подписке. Задача - проанализировать имеющуюся базу данных, содержащую информацию о книгах, издательствах, авторах и пользовательских обзорах для дальнейшего формирования предложений по продукту. 

В рамках работы планируется изучить структуру имеющейся базы, рассмотреть таблицы и произвести ряд расчетов.

Загрузим необходимые библиотеки и подключимся к базе данных.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp', 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Ознакомление с таблицами

Изучим первые строки таблиц, составляющих базу.

## Таблица books

In [27]:
query = '''SELECT * 
FROM books 
LIMIT 1
'''

In [28]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


В таблице **books** хранятся следующие данные:
- уникальный идентификатор книги, 
- уникальный идентификатор автора,
- название книги,
- количество страниц,
- дата публикации,
- уникальный идентификатор издателя

## Таблица authors

In [31]:
query = '''SELECT * 
FROM authors 
LIMIT 1
'''

In [32]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt


В таблице **authors** хранятся следующие данные:
- уникальный идентификатор автора,
- имя автора

## Таблица publishers

In [35]:
query = '''SELECT * 
FROM publishers 
LIMIT 1
'''

In [36]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace


В таблице **publishers** хранятся следующие данные:
- уникальный идентификатор издательства,
- название издательства

## Таблица ratings

In [39]:
query = '''SELECT * 
FROM ratings 
LIMIT 1
'''

In [40]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


В таблице **ratings** хранятся следующие данные:
- уникальный идентификатор оценки,
- уникальный идентификатор книги,
- имя пользователя
- оценка книги

## Таблица reviews

In [43]:
query = '''SELECT * 
FROM reviews 
LIMIT 1
'''

In [44]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


В таблице **reviews** хранятся следующие данные:
- уникальный идентификатор обзора,
- уникальный идентификатор книги,
- имя пользователя
- текст обзора

# Задания

# Задание 1

**Посчитайте, сколько книг вышло после 1 января 2000 года**

In [14]:

query = '''SELECT COUNT(DISTINCT book_id) 
FROM books 
WHERE CAST(DATE_TRUNC('day', publication_date) AS date) >  '2000-01-01'
'''

In [15]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


**Вывод:** общее число вышедших после 1 января 2000 года книг составило 819

# Задание 2
**Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [16]:
query = '''SELECT  b.book_id,
                   b.title,
                   COUNT(DISTINCT rr.review_id) AS reviews,
                   ROUND(AVG(r.rating),2) AS average_rating

FROM books AS b
LEFT OUTER JOIN ratings AS r ON r.book_id = b.book_id
LEFT JOIN reviews AS rr ON rr.book_id = r.book_id
GROUP BY b.book_id

'''

In [17]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,reviews,average_rating
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


# Задание 3

**Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры**

In [18]:
query = '''WITH books_50 AS (SELECT *
FROM books AS b
WHERE num_pages > 50)

SELECT publisher,
       COUNT(book_id) AS books_by_publisher
FROM books_50 AS b50
LEFT JOIN publishers AS p on b50.publisher_id = p.publisher_id
GROUP BY publisher
ORDER BY COUNT(book_id) DESC
LIMIT 1
'''

In [19]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_by_publisher
0,Penguin Books,42


**Вывод:** издательством, выпустившим наибольшее число книг толще 50 страниц стало Penguin Books

# Задание 4

**Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;**

In [20]:
query = '''WITH ratings_50 AS (SELECT b.book_id,
                  b.author_id,
                  COUNT(DISTINCT(rating_id)) AS rating_amount 
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id
HAVING COUNT(DISTINCT(rating_id)) >= 50
ORDER BY COUNT(rating_id) DESC)

SELECT a.author,
       AVG(rr.rating) AS average_rating
FROM ratings_50 AS r
LEFT JOIN authors AS a ON r.author_id = a.author_id
LEFT JOIN ratings AS rr ON r.book_id = rr.book_id
GROUP BY a.author
ORDER BY AVG(rr.rating) DESC
LIMIT 1
'''

In [21]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:** авторами с самой высокой средней оценкой книг, при условии, что книга имеет 50 и более оценок - стали J.K. Rowling и Mary GrandPré

# Задание 5

**Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [22]:
query = '''WITH active_users AS (SELECT username,
                  COUNT(DISTINCT rating_id) ratings_total
FROM ratings AS r
GROUP BY username
HAVING  COUNT(DISTINCT rating_id) > 48
ORDER BY  ratings_total DESC),

active_reviewers AS (SELECT au.username,
       COUNT(DISTINCT review_id) reviews_total
FROM active_users AS au
LEFT JOIN reviews AS r ON au.username = r.username
GROUP BY au.username
ORDER BY COUNT(DISTINCT review_id) DESC)

SELECT AVG(ar.reviews_total)
FROM active_reviewers AS ar
'''

In [23]:
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


**Вывод:** среднее количество обзоров от пользователей, поставивших более 48 оценок составило 24

# Вывод

В рамках работы над проектом была изучена база данных подписочного сервиса для чтения. 

# Ознакомление с данными 

Первым шагом стал импорт библиотек и подключение к базе данных.

Затем было проведено первичное ознакомление с таблицами:

1. Таблица **books** состоит из 1000 строк и содержит следующие данные:
    - уникальный идентификатор книги,
    - уникальный идентификатор автора,
    - название книги,
    - количество страниц,
    - дата публикации,
    - уникальный идентификатор издателя
    

2. Таблица **authors** состоит из 636 строк и хранит следующие данные:
    - уникальный идентификатор автора,
    - имя автора
    
    
3. Таблица **publishers** имеет 340 строк и содержит следующие данные:
    - уникальный идентификатор издательства,
    - название издательства
    
    
4. Таблице **ratings** содержит 6456 строк и хранит следующие данные:

    - уникальный идентификатор оценки,
    - уникальный идентификатор книги,
    - имя пользователя
    - оценка книги
    
5. Таблица **reviews** имеет 2793 строки и содержит следующие данные:

    - уникальный идентификатор обзора,
    - уникальный идентификатор книги,
    - имя пользователя
    - текст обзора
    
# Решение задач

- В ходе первой задачи был дан ответ на вопрос о количестве книг выпущенных после 1 января 2000 года - их число составило 819 единиц


- Результатом второй задачи стало создание таблицы, содержащей:
    - уникальный идентификатор книги,
    - название книги, 
    - количество обзоров на книгу,
    - средний рейтинг книги
    
    
- В рамках третьей задачи было определено издательство, выпустившее наибольшее число книг толще 50 страниц - им стало **Penguin Books,** выпустившее 42 книги, удовлетворяющих обозначенному выше условию 


- Четвертая задача состояла в определении автора с самой высокой средней оценкой книг, при условии, что книга имеет 50 и более оценок - ими стали J.K. Rowling и Mary GrandPré


- В заключительной задаче был дан ответ на вопрос о среднем количестве обзоров от пользователей, поставивших более 48 оценок - их число составило 24